In [46]:
# search for models in "V:\projects\p00542_cpra_2020_lwi_t10\99_import\S3"
import pathlib
import os
import pandas as pd

download_location = r"V:\projects\p00542_cpra_2020_lwi_t10\99_import\S3"

dirs = [x.parent for x in list(pathlib.Path(download_location).rglob('*g??.hdf'))]
# remove duplicates
dirs = sorted(list(set(dirs)))

In [2]:
dirs

[WindowsPath('V:/projects/p00542_cpra_2020_lwi_t10/99_import/S3/Amite_LWI_ReStructuredLikeRegion4/02_Simulation/Amite_RAS'),
 WindowsPath('V:/projects/p00542_cpra_2020_lwi_t10/99_import/S3/Common/deliverables/Example/Amite_LWI/02_Simulation/Amite_RAS'),
 WindowsPath('V:/projects/p00542_cpra_2020_lwi_t10/99_import/S3/lwi-common Example/Models/Amite_RAS'),
 WindowsPath('V:/projects/p00542_cpra_2020_lwi_t10/99_import/S3/Region4/20230822_CTZ_ModelDevelop_DataAnalysis/02_Model/CTZ_08.03.23_Internal_QC'),
 WindowsPath('V:/projects/p00542_cpra_2020_lwi_t10/99_import/S3/Region4/deliverables/20221215_12010004_TO_HMSModelSetup/01_Models/HEC_RAS/BLE_Models/LakeFork/DryCreek'),
 WindowsPath('V:/projects/p00542_cpra_2020_lwi_t10/99_import/S3/Region4/deliverables/20221215_12010004_TO_HMSModelSetup/01_Models/HEC_RAS/BLE_Models/LakeFork/LakeForkCreek'),
 WindowsPath('V:/projects/p00542_cpra_2020_lwi_t10/99_import/S3/Region4/deliverables/20221215_12010004_TO_HMSModelSetup/01_Models/HEC_RAS/BLE_Models/M

In [3]:
# Exclusion terms to remove directoris from list
exclusion_terms = ["BLE_", "_BLE", "hms", "restructuredlike", "example", "backup"]
model_dirs = []
for model in dirs:
    if not any(term.lower() in str(model).lower() for term in exclusion_terms):
        model_dirs.append(model)
    # if any(term.lower() in str(model).lower() for term in exclusion_terms):
        # print (model, "excluded:", term)

To identify the 'best' geometry fo reach model, the following criteria are used:

    1.  Check the highest geometry ##.
    2.  Check the geometry used in the highest plan ##.
    3.  Pull descriptions from each g file.
    4.  For each plan, pull description and which g file is used.

In [30]:
# create empty dataframe with model paths, highest g file, number of g files, highest p file, number of p files.
df = pd.DataFrame(columns=["model_path", "region", "project_name", "g_hi", "g_cnt", "p_hi", "p_cnt"])

# iterate each model directory.
for model_path in reversed(model_dirs):
    region = str(model_path).split("\\")[5]
    # Get highest g## file in each model directory.
    g_file_paths = list(pathlib.Path(model_path).glob('*.g??'))
    # remove '.gdb' paths from list
    g_file_paths = [x for x in g_file_paths if '.gdb' not in str(x)]
    g_files = [x.name for x in g_file_paths]
    # g_file_paths_str = [str(x) for x in g_file_paths]
    g_files.sort()
    if len(g_files) == 0:
        print("No g## files found in", model_path, '\nremoving from model list')
        g_hi = None
        model_dirs.remove(model_path)
        continue
    elif len(g_files) == 1:
        # print("Only one g## file found in", model_path)
        g_hi = g_files[0]
    else:
        g_hi = g_files[-1]
    
    # if g_hi.lower() contains 'backup' remove from mode_dirs list.
    if 'backup' in g_hi.lower():
        print(g_hi, "includes the exclusion word 'backup'. \nFound in", model_path, "\nremoving from model list")
        model_dirs.remove(model_path)
        continue 

    # Get highest p## file in each model directory.
    p_file_paths = list(pathlib.Path(model_path).glob('*.p??'))
    # remove '.prj' paths from list
    p_file_paths = [x for x in p_file_paths if '.prj' not in str(x)]
    p_file_paths = [x for x in p_file_paths if '.Backup' not in str(x)]
    p_files = [x.name for x in p_file_paths]
    p_files.sort()
    if len(p_files) == 0:
        print("No p## files found in", model_path)
        p_hi = None
        continue
    elif len(p_files) == 1:
        # print("Only one p## file found in", model_path)
        p_hi = p_files[0]
    else:
        p_hi = p_files[-1]
    
    # if g_hi.lower() contains 'backup' remove from mode_dirs list.
    if 'backup' in g_hi.lower():
        print(p_hi, "includes the exclusion word 'backup'. \nFound in", model_path, "\nremoving from model list")
        model_dirs.remove(model_path)
        continue 

    project_name = g_hi.split('.')[0:-1]
    project_name = '.'.join(project_name)
    # if project_name.lower() contains 'backup' remove from mode_dirs list.
    if 'backup' in project_name.lower():
        print(project_name, "includes the exclusion word 'backup'. \nFound in", model_path, "\nremoving from model list")
        model_dirs.remove(model_path)
        continue

    # columns=["model_path", "project_name", "g_hi", "g_cnt", "p_hi", "p_cnt"])
    df.loc[len(df.index)] = [str(model_path), region, project_name, g_hi, len(g_files), p_hi, len(p_files)] 

# view list of highest g## files


In [31]:
df

,model_path,region,project_name,g_hi,g_cnt,p_hi,p_cnt
0,V:\projects\p00542_cpra_2020_lwi_t10\99_import...,Region7,TickfawCrk,TickfawCrk.g07,1,TickfawCrk.p11,1
1,V:\projects\p00542_cpra_2020_lwi_t10\99_import...,Region7,Tickfaw,Tickfaw.g04,2,Tickfaw.p07,2
2,V:\projects\p00542_cpra_2020_lwi_t10\99_import...,Region7,Tickfaw_Natalbany,Tickfaw_Natalbany.g05,2,Tickfaw_Natalbany.p07,2
3,V:\projects\p00542_cpra_2020_lwi_t10\99_import...,Region7,MT,MT.g05,1,MT.p12,1
4,V:\projects\p00542_cpra_2020_lwi_t10\99_import...,Region7,LizardCrk2,LizardCrk2.g04,2,LizardCrk2.p15,4
...,...,...,...,...,...,...,...
73,V:\projects\p00542_cpra_2020_lwi_t10\99_import...,Region4,WH_Whisky_Chitto,WH_Whisky_Chitto.g17,2,WH_Whisky_Chitto.p19,3
74,V:\projects\p00542_cpra_2020_lwi_t10\99_import...,Region4,LC_LowerCalcasieu,LC_LowerCalcasieu.g01,1,LC_LowerCalcasieu.p08,3
75,V:\projects\p00542_cpra_2020_lwi_t10\99_import...,Region4,WF_WestForkCalcasieu,WF_WestForkCalcasieu.g01,2,WF_WestForkCalcasieu.p03,4
76,V:\projects\p00542_cpra_2020_lwi_t10\99_import...,Region4,UC_UpperCalcasieu,UC_UpperCalcasieu.g01,1,UC_UpperCalcasieu.p40,3


In [32]:
df['duplicate'] = df.project_name.duplicated(keep=False)

df.to_csv("model_list.csv", index=False)

In [33]:
df_has_duplicates = df.project_name.drop_duplicates()
df_has_duplicates.to_csv("model_list_has_duplicates.csv", index=False)

In [20]:
df

,model_path,region,project_name,g_hi,g_cnt,p_hi,p_cnt,duplicate
0,V:\projects\p00542_cpra_2020_lwi_t10\99_import...,Region7,TickfawCrk,TickfawCrk.g07,1,TickfawCrk.p11,1,False
1,V:\projects\p00542_cpra_2020_lwi_t10\99_import...,Region7,Tickfaw,Tickfaw.g04,2,Tickfaw.p07,2,False
2,V:\projects\p00542_cpra_2020_lwi_t10\99_import...,Region7,Tickfaw_Natalbany,Tickfaw_Natalbany.g05,2,Tickfaw_Natalbany.p07,2,False
3,V:\projects\p00542_cpra_2020_lwi_t10\99_import...,Region7,MT,MT.g05,1,MT.p12,1,False
4,V:\projects\p00542_cpra_2020_lwi_t10\99_import...,Region7,LizardCrk2,LizardCrk2.g04,2,LizardCrk2.p15,4,False
...,...,...,...,...,...,...,...,...
73,V:\projects\p00542_cpra_2020_lwi_t10\99_import...,Region4,WH_Whisky_Chitto,WH_Whisky_Chitto.g17,2,WH_Whisky_Chitto.p19,3,False
74,V:\projects\p00542_cpra_2020_lwi_t10\99_import...,Region4,LC_LowerCalcasieu,LC_LowerCalcasieu.g01,1,LC_LowerCalcasieu.p08,3,True
75,V:\projects\p00542_cpra_2020_lwi_t10\99_import...,Region4,WF_WestForkCalcasieu,WF_WestForkCalcasieu.g01,2,WF_WestForkCalcasieu.p03,4,True
76,V:\projects\p00542_cpra_2020_lwi_t10\99_import...,Region4,UC_UpperCalcasieu,UC_UpperCalcasieu.g01,1,UC_UpperCalcasieu.p40,3,True


# Request has been sent to each Region to review these findings and provide:

1.	Review spreadsheet of model inventory to ensure all your models are there.
2.	Fill in Column J to specify the best geometry file number (Ex: g.01).

The spreadsheet sent is: "./model_list.xlsx"

Once it has been returned, we will have a final list of models and geometry files to use for the next steps.


In [ ]:
# Read each best g file in returned excel sheet.
# Get desctiption, and structures.
# Map structures with descriptions as popups.
# Create merged g file.

In [40]:
# Based on best data now, open excel sheet and get non-duplicate project names and accositated best g file.
df = pd.read_excel("model_list.xlsx", sheet_name="model_list")
df = df[df["Region Specified Best Geoemtry File"].notnull()]

In [47]:
# For each g file, get description and structures.
# for model in df.iterrows():
#     # get g file path
#     g_num = model[1]["Region Specified Best Geoemtry File"]
#     project_name = model[1]["Project Name"]
#     g_path = f'{download_location}\\{model[1]["Project Directory"]}\\{project_name}.{g_num.replace(".", "")}'


In [49]:
df["g_path"] = df.apply(lambda x: f'{download_location}\\{x["Project Directory"]}\\{x["Project Name"]}.{x["Region Specified Best Geoemtry File"].replace(".", "")}', axis=1)

In [72]:
# For each g_path, get description and structures.
for model in df.iterrows():
    project_name = model[1]["Project Name"]
    with open(model[1]["g_path"]) as g_path:
        g_file = g_path.readlines()
    # Remove new line characters.
    g_file = [x.replace('\n','') for x in g_file]

In [80]:
# for testing 
with open(r"V:\projects\p00542_cpra_2020_lwi_t10\99_import\S3\Region7\deliverables\BST_RAS_Backcheck\RAS\BS\6_3\BayouSara.g02") as g_path:
    g_file = g_path.readlines()
# Remove new line characters.
g_file = [x.replace('\n','') for x in g_file]

In [85]:
# Split key values from g file.
g_kv = [x.split('=') for x in g_file]

# Remove empty strings from key value pairs.
g_kv = [x for x in g_kv if len(x) == 2]

# Remove pairs where the value is empty or NaN.
g_kv = [x for x in g_kv if x[1] is not None and x[1] != 'nan' and x[1] != '' and x[1] is not None]

# # create dictionary from key value pairs.
# g_kv = dict(g_kv)


In [87]:
# get unique keys from g file.
g_keys = sorted(list(set([x[0] for x in g_kv])))

In [ ]:
# get description
